In [ ]:
# import libraries
import pandas as pd
import fuzzywuzzy
import chardet
import numpy as np
import datetime

In [ ]:
# import data
store_df = pd.read_csv('store_income_data_task.csv', keep_default_na=True)
print(store_df)

In [ ]:
# Here, I am taking a look at all the unique values in the "country" column. 
# Then, I will convert the column entries to lowercase and remove any trailing white spaces.
store_df['country'] = store_df['country'].str.lower()
store_df['country'] = store_df['country'].str.strip()
countries = store_df['country'].unique()
print(f"There are {len(countries)} unique countries")
print(countries)



In [ ]:
# Cleaning up the country column
# Finding the closet matches for each country as I am trying to narrow it down to three countries
# The three countries are "united kingdom", "united states" and "south africa"
from fuzzywuzzy import process
uk_matches = fuzzywuzzy.process.extract("united kingdom", countries, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(uk_matches)
us_matches = fuzzywuzzy.process.extract("united states", countries, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(us_matches)
sa_matches = fuzzywuzzy.process.extract("south africa", countries, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(sa_matches)

In [ ]:
# Defining the function to replace the countries in the column
def replace_matches_in_column(store_df, column, string_to_match, min_ratio=90):
    strings = store_df[column].unique()
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=3, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_close_matches = store_df[column].isin(close_matches)
    store_df.loc[rows_with_close_matches, column] = string_to_match
    print("Done!")

In [ ]:
# Since there are many different variations of how the country names are written in this dataset
# I have called the replace_matches_in_column function quite a lot fo times to narrow it down
# So that it is easier replace the different names with the desired country name later
replace_matches_in_column(store_df, column='country', string_to_match="united kingdom")
replace_matches_in_column(store_df, column='country', string_to_match="united states")
replace_matches_in_column(store_df, column='country', string_to_match="south africa")
replace_matches_in_column(store_df, column='country', string_to_match="united states of america")
replace_matches_in_column(store_df, column='country', string_to_match="britain")
replace_matches_in_column(store_df, column='country', string_to_match="s.africasouth africa")
replace_matches_in_column(store_df, column='country', string_to_match="uk")
replace_matches_in_column(store_df, column='country', string_to_match="england")
replace_matches_in_column(store_df, column='country', string_to_match="america")
# Checking to see how many different countries names there are to see if the functions worked properly
countries = store_df['country'].unique()
print(countries)

In [ ]:
# to finish cleaning the country column, I am now going to use the replace() function
# This will help me get rid of the different variations and change them into the desired country names
store_df.replace(['uk', 'britain','england', 'u.k.', 'u.k/', 'u.k'], 'united kingdom', inplace=True)
store_df.replace(['united states of america', 'america'], 'united states', inplace=True)
store_df.replace(['sa', 's.a.', 's.africasouth africa', 'sa/', 's.a..', 's.a./', 'sa.'], 'south africa', inplace=True)
countries = store_df['country'].unique()
# For the missing or random values below, I will also make them consistent
# To replace NaN, I have used numpy because of the way missing values are presented in panda
store_df.replace(['', '/', '.', np.nan], 'N/A', inplace=True)
# Dropping the the rows with 'N/A' to get the desired output
store_df = store_df[store_df['country'] != 'N/A']
# Printing out 'countries' and 'store_df' to double check that there are only three unique countries
print(countries)
print(store_df)


Create a new column called `days_ago` in the DataFrame that is a copy of
the “date_measured” column but instead shows a number that represents the number of days ago that it was measured. Note that the current date can be obtained using datetime.date.today().

In [ ]:
# Converting the 'date_measured' column to datetime format
store_df['date_measured'] = pd.to_datetime(store_df['date_measured'], format='%d-%m-%Y')
print(store_df['date_measured'])
# Creating a new column for 'days_ago'
from datetime import date
# In order to calculate the number of days ago it was measured, I have written the equation below
# I researched and realised I can use the Timestamp funciton in pandas to help me do so
# I have converted the datetime.date.today into a Timestamp object to help do the calculations
# I have named the Timestamp object as the variable today
# I also used the dt.days to turn the result into number of days
# The website I used is here: https://www.geeksforgeeks.org/python-pandas-timestamp-now/
today = pd.Timestamp(datetime.date.today())
store_df['days_ago'] = (today - store_df['date_measured']).dt.days
print(store_df['days_ago'])